In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./outputs/combined/ias-clean.csv")
# df = df.drop(columns=["Unnamed 0:"])

In [3]:
df

,Name,Service,Cadre,Allotment Year,Date of Birth,Date of Joining,Identity No.,Source of Recruitment,Gender,Place of Domicile,...,Qualification/University/Institute,Subject,Division,Designation/Level,Ministry/Department/Office/Location,Organisation,Experience(major/minor),Period(From/To),Dataset name,Clean Ids
0,Shri B G Deshmukh,Indian Administrative Services,Maharashtra,1951,26/03/1929,01/07/1951,02MH006800,Direct Recruitment,Male,Maharashtra,...,Graduate,Botany,Ordinary,N.A./Secretary|OSD to Minister/Secretary|Secre...,P M O|Cabinet Sectt|M/o Labour & Rehabilitatio...,Centre|Centre|Centre|Centre|Cadre (AIS)|Cadre ...,General Administration / Personnel and General...,01/03/1989 - 01/12/1990|01/08/1986 - 01/08/198...,Supremo,MH006800
1,Shri Gv Ramakrishna,Indian Administrative Services,Andhra Pradesh,1952,27/03/1930,01/07/1952,01AP004800,Direct Recruitment,Male,Karnataka,...,Graduate|P.G.,Chemistry|Chemistry,First|First,N.A./Secretary|Secretary/Secretary Equivalent|...,|M/o Home Affairs/D/o Home/Commission on Centr...,Cadre (Inter-Cadre Deputation)|Centre|Cadre (A...,Personnel Mgmt / Personnel and General Adminis...,01/04/1988 - 01/01/1989|07/05/1984 - 14/02/198...,Supremo,AP004800
2,Shri N Nagamani,Indian Administrative Services,Bihar,1952,06/05/1930,01/07/1952,01BH005400,Direct Recruitment,Male,Tamil Nadu,...,Graduate|P.G.,Mathematics|Statistics,First|First,N.A./Secretary|N.A./Additional Secretary|Commi...,|||||||Saran||,Cadre (AIS)|Cadre (AIS)|Cadre (AIS)|Cadre (AIS...,Planning / Planning & Prog Implementation|Deve...,01/08/1984 - 31/05/1988|01/08/1981 - 01/07/198...,Supremo,BH005400
3,Shri Pp Nayyar,Indian Administrative Services,Bihar,1952,07/07/1930,01/07/1952,01BH005600,Direct Recruitment,Male,Uttar Pradesh,...,Graduate|P.G.,Physics|Mathematics,First|First,OSD to Minister/Secretary Equivalent|Chairman/...,M/o Home Affairs/D/o Home|Cabinet Sectt|||M/o ...,Centre|Centre|Cadre (AIS)|Cadre (AIS)|Centre|C...,Home / Home|Internal Security / Home|General A...,01/12/1987 - 01/07/1988|01/08/1983 - 01/07/198...,Supremo,BH005600
4,Shri M Sivagnanam,Indian Administrative Services,Gujarat,1952,23/02/1930,01/07/1952,01GJ003000,Direct Recruitment,Male,Tamil Nadu,...,Graduate|P.G.,Economics|Economics,Second|Second,Managing Director/Secretary|Chief Secy/Secreta...,|||Planning Commission|||||,Cadre (AIS)|Cadre (AIS)|Cadre (AIS)|Centre|Cad...,Corporate Management (New) / Corporate Managem...,01/08/1987 - 01/02/1988|01/12/1984 - 01/07/198...,Supremo,GJ003000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14892,Ms. Tamil Oviya S,Indian Administrative Services,West Bengal,2019,26-09-1993,01-09-2019,WB118X08,RR,N.A.,Tamil Nadu,...,Graduate,N.A.,N.A.,N.A.,N.A.,N.A.,N.A.,01-09-2018,DOPT(easynic),WB118X08
14893,Ms. Renu Sogan,Indian Administrative Services,West Bengal,2019,21-03-1994,01-09-2019,WB118X09,RR,N.A.,Rajasthan,...,Graduate,N.A.,N.A.,N.A.,N.A.,N.A.,N.A.,01-09-2018,DOPT(easynic),WB118X09
14894,Shri Anuj Pratap Singh,Indian Administrative Services,West Bengal,2019,30-03-1991,01-09-2019,WB118X10,RR,N.A.,Uttar Pradesh,...,Graduate,N.A.,N.A.,N.A.,N.A.,N.A.,N.A.,01-09-2018,DOPT(easynic),WB118X10
14895,Ms. Priyanka Singh,Indian Administrative Services,West Bengal,2019,25-03-1992,01-09-2019,WB118X11,RR,N.A.,Uttar Pradesh,...,Graduate,N.A.,N.A.,N.A.,N.A.,N.A.,N.A.,01-09-2018,DOPT(easynic),WB118X11


In [4]:
for column_name in df.columns:
    for i in df.index:
        if df[column_name][i] == "Not Found" or df[column_name][i] == "N.A./Not Available" or df[column_name][i] == "Not Available"  or df[column_name][i] == "N. A." or str(df[column_name][i]) == "nan":
            df[column_name][i] = "N.A."

In [5]:
for i,value in enumerate(df["Identity No."]):
    if str(value) == "nan":
        print(i)

In [6]:
id_values = df["Clean Ids"].values

In [7]:
from collections import Counter
id_counter = Counter(id_values)

In [8]:
repeats = [item for item, count in id_counter.items() if count > 2 and not item == "N.A."]

In [9]:
repeats_df = pd.DataFrame(columns=df.columns)
for id_value in repeats:
    rows = df.loc[df["Clean Ids"] == id_value]
    repeats_df = repeats_df.append(rows, ignore_index=True)

In [10]:
repeats_df.to_csv("repeats.csv", index=False)

In [10]:
education_columns = ["Qualification/University/Institute", "Subject", "Division"]
background_columns = ["Designation/Level", "Ministry/Department/Office/Location" ,"Organisation", "Experience(major/minor)", "Period(From/To)"]

In [11]:
rows = df.loc[df["Clean Ids"] == repeats[0]]
# rows = rows.drop(rows.loc[rows["Dataset name"] == "DOPT(easynic)"].index)

In [12]:
import math
def make_verbose(start_with, done, total, max_lines=25):
    num_lines = int((done/total)*max_lines)
    lines = "="*num_lines
    spaces = " "*(max_lines-num_lines)
    print(f"\r{start_with}  {lines}>{spaces}  {done}/{total}", end="", sep=" ", flush=True)

In [13]:
# check education
def combine_rows(rows):
    my_dict = {}
    if len(rows.index) > 1:
        rows = rows.drop(rows.loc[rows["Dataset name"] == "DOPT(easynic)"].index)
        if len(rows.index) > 1:
            if not rows["Name"].values[0] == rows["Name"].values[1]:
                return rows
            for column in rows.columns:
                selector = 0
                if (str(column) in education_columns or str(column) in background_columns):
                    if len(str(rows.iloc[0][column])) <= len(str(rows.iloc[1][column])):
                        selector = 1

                my_dict[column] = str(rows.iloc[selector][column]).replace("-", "/")
        else:
            index = rows.index[0]
            my_dict = dict(rows.loc[index])
    else:
        index = rows.index[0]
        my_dict = dict(rows.loc[index])
    return my_dict

In [16]:
pd.DataFrame(combine_rows(df.loc[df["Clean Ids"] == "RJ029800"]), index=[0])

,Name,Service,Cadre,Allotment Year,Date of Birth,Date of Joining,Identity No.,Source of Recruitment,Gender,Place of Domicile,...,Qualification/University/Institute,Subject,Division,Designation/Level,Ministry/Department/Office/Location,Organisation,Experience(major/minor),Period(From/To),Dataset name,Clean Ids
0,Shri Rajeev Singh Thakur,Indian Administrative Services,Rajasthan,1995,29/06/1969,03/09/1995,01RJ029800,Direct Recruitment,Male,Madhya Pradesh,...,M.A./BARKATULLAH UNIVERSITY BHOPAL BHOPAL/Cent...,Physics|Economics,Second|First,Joint Secretary/Joint Secretary|Secretary to G...,M/o Defence/Department of Military Affairs/New...,Centre|Cadre (AIS)|Centre|Centre|Cadre (AIS)|C...,Defence / Defence|Consumer Affairs / Consumer ...,29/01/2020 / 03/12/2022|08/05/2017 / 03/12/201...,Supremo,RJ029800


In [15]:
test = df.loc[df["Clean Ids"] == "JH033100"]
combine_rows(test)

{'Name': 'Shri Rajiv Gauba',
 'Service': 'Indian Administrative Services',
 'Cadre': 'Jharkhand ',
 'Allotment Year': '1982',
 'Date of Birth': '15/08/1959',
 'Date of Joining': '01/09/1982',
 'Identity No.': '01JH033100',
 'Source of Recruitment': 'Direct Recruitment',
 'Gender': 'Male',
 'Place of Domicile': 'Bihar',
 'Mother Tongue': 'Punjabi',
 'Languages Known': 'English Hindi',
 'Retirement Reason': 'Serving',
 'Qualification/University/Institute': 'Graduate',
 'Subject': 'Physics',
 'Division': 'First',
 'Designation/Level': 'Cabinet Secretary/Cabinet Secretary|Secretary/Secretary|Secretary/Secretary|N.A./Additional Secretary|Additional Secy/Additional Secretary|Resident Commissioner/Additional Secretary|N.A./Joint Secretary|On Leave/Director Equivalent|Pvt Secy/Director Equivalent|Pvt Secy/Director Equivalent|Pvt Secy/Deputy Secretary Equivalent|Collector & D M/Deputy Secretary|Collector & D M/Under Secretary|C E O/Under Secretary|Deputy Dev Commissioner/Junior Scale',
 'Minist

In [16]:
df = pd.read_csv("./outputs/combined/ias-clean.csv")
na_ids = df.loc[df["Clean Ids"] == "N.A."]
discrepency = pd.DataFrame(columns=df.columns)
df = df.drop(na_ids.index)

In [17]:
id_counter = Counter(df["Clean Ids"].values)
new_df = pd.DataFrame(columns=df.columns)
length = len(id_counter)
for i, id_val in enumerate(id_counter):
    make_verbose(f"{id_val} {id_counter.get(id_val, 0)}", i+1, length)
    rows = df.loc[df["Clean Ids"] == str(id_val)]
    new_row = combine_rows(rows)
    new_df = new_df.append(new_row, ignore_index=True)

WB118X12 1  =========================>  11567/11567

In [18]:
new_df = new_df.drop(columns=["Identity No."])

In [19]:
new_df.to_csv("ias-clean-combined.csv", index=False)

In [20]:
year_indexes = new_df.loc[new_df["Allotment Year"] == "####"].index

In [21]:
length = len(year_indexes)
for counter, i in enumerate(year_indexes):
    make_verbose(i, counter, length)
    date_of_joining = str(new_df["Date of Joining"][i])
    year_of_joining = date_of_joining.replace("-","/").split("/")[2]
    new_df["Allotment Year"][i] = year_of_joining

11166  ========================>   114/115

In [23]:
for counter, i in enumerate(new_df.index):
    make_verbose(i, counter, len(new_df.index))
    date_of_joining = str(new_df["Date of Joining"][i])
    date_of_birth = str(new_df["Date of Birth"][i])
    date_of_birth = date_of_birth.replace("-", "/")
    date_of_joining = date_of_joining.replace("-", "/")
    new_df["Date of Joining"][i] = date_of_joining
    new_df["Date of Birth"][i] = date_of_birth

11568  ========================>   11568/11569

In [24]:
new_df.loc[new_df["Date of Birth"] == new_df["Date of Joining"]]

,Name,Service,Cadre,Allotment Year,Date of Birth,Date of Joining,Source of Recruitment,Gender,Place of Domicile,Mother Tongue,...,Qualification/University/Institute,Subject,Division,Designation/Level,Ministry/Department/Office/Location,Organisation,Experience(major/minor),Period(From/To),Dataset name,Clean Ids
2560,Shri Basant Kishor Bal,Indian Administrative Services,Manipur,1979,N.A.,N.A.,N.A.,Male,N.A.,N.A.,...,N.A.,N.A.,N.A.,N.A.,N.A.,Cadre (AIS),/ N.Applicable/N.Available,-,Supremo,MT007500
4687,Shri K Rama Krishna,Indian Administrative Services,Andhra Pradesh,1990,26/04/1939,26/04/1939,By Promotion from State Civil Service,Male,Andhra Pradesh,Telugu,...,Graduate,N.A.,Ordinary,N.A./Director,N.A.,Cadre (AIS),Languages / Human Resource Dev,01/01/1995 - 30/04/1997,Supremo,AP909110
9210,Ms. Varsha Singh,Indian Administrative Services,Bihar,2016,N.A.,N.A.,Direct Recruitment,Female,Bihar,N.A.,...,B.Sc./Nagpur Nagpur/Nagpur University College,Science,First,Assistant Secy/Junior Scale|N.A./Not Available,M/o Human Resources Dev/New Delhi|,Centre|Cadre (AIS),School Education / Human Resource Dev|/ N.Appl...,02/07/2018 - 28/09/2018|-,Supremo,BH115D05
9246,Shri Mali Rahul Bhanudas,Indian Administrative Services,Nagaland,2016,N.A.,N.A.,Direct Recruitment,Male,Maharashtra,N.A.,...,B.E./Shivaji Univ. Kohlapur/University College,Prodn.Engg.,First,S D O (C)/Junior Scale|On Training/Junior Scale,|,Cadre (AIS)|Cadre (AIS),District Admn / Land Revenue Mgmt & District A...,10/10/2018 -|01/09/2016 - 01/07/2018,Supremo,NL115P04


In [29]:
new_df.to_csv("./outputs/combined/ias-clean-final.csv", index=False)